#### Imports

In [1]:
import pandas as pd
import json
from sqlalchemy import create_engine, Column, Integer, String, Sequence, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
import pymysql
from tqdm.notebook import tqdm

## Conhecendo os dados

In [2]:
file = open('logs_1.txt', 'r')
json_list_1 = file.readlines()
file.close()
file = open('logs_2.txt', 'r')
json_list_2 = file.readlines()
file.close()
json_list = json_list_1 + json_list_2
print(json_list[0])

{"request":{"method":"GET","uri":"\/","url":"http:\/\/cole.com","size":80,"querystring":[],"headers":{"accept":"*\/*","host":"cole.com","user-agent":"curl\/7.37.1"}},"upstream_uri":"\/","response":{"status":401,"size":585,"headers":{"Content-Length":"197","via":"kong\/1.3.0","Connection":"close","access-control-allow-credentials":"true","Content-Type":"application\/json","server":"nginx","access-control-allow-origin":"*"}},"authenticated_entity":{"consumer_id":{"uuid":"631faf11-30ec-355b-bdae-a917c9518a84"}},"route":{"created_at":1564823899,"hosts":"miller.com","id":"0636a119-b7ee-3828-ae83-5f7ebbb99831","methods":["GET","POST","PUT","DELETE","PATCH","OPTIONS","HEAD"],"paths":["\/"],"preserve_host":false,"protocols":["http","https"],"regex_priority":0,"service":{"id":"c3e86413-648a-3552-90c3-b13491ee07d6"},"strip_path":true,"updated_at":1564823899},"service":{"connect_timeout":60000,"created_at":1563589483,"host":"ritchie.com","id":"c3e86413-648a-3552-90c3-b13491ee07d6","name":"ritchie

In [3]:
data = json.loads(json_list[0])
print('usuario')
print(data["authenticated_entity"]["consumer_id"]["uuid"])
print('----------------------------')
print('serviço')
print(data["service"]["id"])
print(data["service"]["host"])
print(data["service"]["name"])
print('----------------------------')
print('latencias')
print(data["latencies"]["proxy"])
print(data["latencies"]["kong"])
print(data["latencies"]["request"])
print('----------------------------')
print('requisicao')
print(data["request"]["headers"]["host"])
print('----------------------------')
serie = {
    'consumer_id':data["authenticated_entity"]["consumer_id"]["uuid"],
    'service_id':data["service"]["id"],
    'proxy':data["latencies"]["proxy"],
    'kong':data["latencies"]["kong"],
    'request':data["latencies"]["request"],
    'host':data["service"]["host"],
    'name':data["service"]["name"],
    'request_host':data["request"]["headers"]["host"]
    }
print(serie)

usuario
631faf11-30ec-355b-bdae-a917c9518a84
----------------------------
serviço
c3e86413-648a-3552-90c3-b13491ee07d6
ritchie.com
ritchie
----------------------------
latencias
1325
16
2192
----------------------------
requisicao
cole.com
----------------------------
{'consumer_id': '631faf11-30ec-355b-bdae-a917c9518a84', 'service_id': 'c3e86413-648a-3552-90c3-b13491ee07d6', 'proxy': 1325, 'kong': 16, 'request': 2192, 'host': 'ritchie.com', 'name': 'ritchie', 'request_host': 'cole.com'}


In [4]:
df = pd.DataFrame(columns = ['consumer_id','service_id','proxy','kong','request','host','name','request_host'])
df = df.append(serie,ignore_index=True)
df.head()

,consumer_id,service_id,proxy,kong,request,host,name,request_host
0,631faf11-30ec-355b-bdae-a917c9518a84,c3e86413-648a-3552-90c3-b13491ee07d6,1325,16,2192,ritchie.com,ritchie,cole.com


## Extraindo dados relevantes

In [5]:
%%time
df = pd.DataFrame(columns = ['consumer_id','service_id','proxy','kong','request','host','name','request_host'])
serie_list = []
for json_str in json_list:
    data = json.loads(json_str)
    serie = {
    'consumer_id':data["authenticated_entity"]["consumer_id"]["uuid"],
    'service_id':data["service"]["id"],
    'proxy':data["latencies"]["proxy"],
    'kong':data["latencies"]["kong"],
    'request':data["latencies"]["request"],
    'host':data["service"]["host"],
    'name':data["service"]["name"],
    'request_host':data["request"]["headers"]["host"]
    }
    serie_list.append(serie)
    
df = df.append(serie_list,ignore_index=True)
print(len(df))
df.index += 1
df.head()

100000
Wall time: 4.33 s


,consumer_id,service_id,proxy,kong,request,host,name,request_host
1,631faf11-30ec-355b-bdae-a917c9518a84,c3e86413-648a-3552-90c3-b13491ee07d6,1325,16,2192,ritchie.com,ritchie,cole.com
2,3de80f20-4109-37f0-85af-7a920384e3eb,d035ffcf-914a-3007-b028-ae18f04d75b4,1686,17,1891,terry.org,terry,kertzmann.com
3,3b6fdb67-1049-377a-9d46-6538aa8cc1e2,a5bf08bd-c030-30d5-8009-83a8c30103bf,1105,12,1059,orn.com,orn,lowe.com
4,e49e2600-ce15-3004-a1b0-129b84fda7f8,22f8e3a6-01f7-3264-b4b5-9d178df11d06,937,13,2279,sauer.com,sauer,abbott.biz
5,2c69f722-b348-3f44-8b26-3a409dcb0396,c3e86413-648a-3552-90c3-b13491ee07d6,1388,20,1209,ritchie.com,ritchie,lubowitz.com


## Aplicando transformações nos dados

### Tabela de serviços

In [6]:
servicos = df.drop_duplicates(subset=['service_id']).reset_index(drop=True)\
.drop(['consumer_id','proxy','kong','request','request_host'], axis=1)
servicos.index += 1
servicos_map = servicos['service_id'].to_dict()
servicos_inv_map = {v: k for k, v in servicos_map.items()}
servicos.head()

,service_id,host,name
1,c3e86413-648a-3552-90c3-b13491ee07d6,ritchie.com,ritchie
2,d035ffcf-914a-3007-b028-ae18f04d75b4,terry.org,terry
3,a5bf08bd-c030-30d5-8009-83a8c30103bf,orn.com,orn
4,22f8e3a6-01f7-3264-b4b5-9d178df11d06,sauer.com,sauer
5,eb1ce287-5797-3e45-b9a6-e6d51691257e,corkery.com,corkery


### Tabela de Consumidores

In [7]:
costumer = df.drop_duplicates(subset=['consumer_id']).reset_index(drop=True)\
.drop(['service_id','proxy','kong','request','host','name','request_host'], axis=1)
costumer.index += 1
print(len(costumer))
costumer_map = costumer['consumer_id'].to_dict()
costumer_inv_map = {v: k for k, v in costumer_map.items()}
costumer.head()

9999


,consumer_id
1,631faf11-30ec-355b-bdae-a917c9518a84
2,3de80f20-4109-37f0-85af-7a920384e3eb
3,3b6fdb67-1049-377a-9d46-6538aa8cc1e2
4,e49e2600-ce15-3004-a1b0-129b84fda7f8
5,2c69f722-b348-3f44-8b26-3a409dcb0396


### Tabela de Requisições

In [8]:
%%time
requisicoes = df.replace({"consumer_id": costumer_inv_map, "service_id": servicos_inv_map})\
.drop(['host','name'], axis=1)
print(len(costumer))
requisicoes.head()

9999
Wall time: 1min 43s


,consumer_id,service_id,proxy,kong,request,request_host
1,1,1,1325,16,2192,cole.com
2,2,2,1686,17,1891,kertzmann.com
3,3,3,1105,12,1059,lowe.com
4,4,4,937,13,2279,abbott.biz
5,5,1,1388,20,1209,lubowitz.com


## Carregando dados no banco

### Criando tabelas no sevidor

In [9]:
user = 'test:1234'
dbPath = 'mysql+pymysql://'+ user +'@127.0.0.1:3306/test'
sqlEngine = create_engine(dbPath, pool_recycle=3600)
dbConnection = sqlEngine.connect()
Base = declarative_base()

class Consumers(Base):
    __tablename__="consumers"
    index =Column(Integer, Sequence('index_seq'), primary_key=True)
    consumer_id = Column(String(200))
    
class Services(Base):
    __tablename__="services"
    index= Column(Integer, Sequence('index_seq'), primary_key=True)
    service_id = Column(String(200))
    host = Column(String(100))
    name = Column(String(100))

class Requests(Base):
    __tablename__ = "requests"
    index = Column(Integer, Sequence('index_seq'), primary_key=True)
    consumer_id = Column(Integer, ForeignKey(Consumers.index))
    service_id = Column(Integer,ForeignKey(Services.index))
    proxy = Column(Integer)
    kong = Column(Integer)
    request = Column(Integer)
    request_host = Column(String(100))
    consumer = relationship('Consumers', foreign_keys='Consumers.index')
    Service = relationship('Services', foreign_keys='Services.index')

Consumers.__table__.create(bind=sqlEngine, checkfirst=True)
Services.__table__.create(bind=sqlEngine, checkfirst=True) 
Requests.__table__.create(bind=sqlEngine, checkfirst=True)
dbConnection.close()

### Colocanda dados nas tabelas

In [10]:
try:
    sqlEngine = create_engine(dbPath, pool_recycle=3600)
    dbConnection = sqlEngine.connect()
    frame = costumer.to_sql("consumers", dbConnection, if_exists='append')
    frame = servicos.to_sql("services", dbConnection, if_exists='append')
    frame = requisicoes.to_sql("requests", dbConnection, if_exists='append');
except ValueError as vx:
    print(vx)
except Exception as ex:   
    print(ex)
else:
    print("Tabelas criadas");   
finally:
    dbConnection.close()

Tabelas criadas


## Gerando Relatórios Requisitados

### Requisições por consumidor

In [11]:
sqlEngine = create_engine(dbPath, pool_recycle=3600)
dbConnection = sqlEngine.connect()
consumers = pd.read_sql_query('select * from consumers', dbConnection, index_col='index')
for consumer in tqdm(consumers.iterrows(), total=consumers.shape[0]):
    querry = 'select \
                r.index,\
                r.proxy,\
                r.kong,\
                r.request,\
                r.request_host,\
                s.service_id \
            from \
            test.requests as r \
            inner join test.services as s \
            on r.service_id = s.index \
            and r.service_id = ' + str(consumer[0])
    pd.read_sql_query(querry, dbConnection, index_col='index').to_csv('consumers/' + consumer[1][0] +'.csv')
dbConnection.close()
print('Requsições por consumidor salvas em consumers/')


Requsições por consumidor salvas em consumers/


### Requisições por serviço

In [12]:
sqlEngine = create_engine(dbPath, pool_recycle=3600)
dbConnection = sqlEngine.connect()
services = pd.read_sql_query('select * from services', dbConnection, index_col='index')
for service in tqdm(services.iterrows(), total=services.shape[0]):
    querry = 'select \
                r.index,\
                r.proxy,\
                r.kong,\
                r.request,\
                r.request_host,\
                s.service_id \
            from \
                test.requests as r \
            inner join test.services as s \
            on r.service_id = s.index \
            and r.service_id = ' + str(service[0])
    pd.read_sql_query(querry, dbConnection, index_col='index').to_csv('services/' + service[1][0] +'.csv')
dbConnection.close()
print('Requsições por serviço salvas em services/')


Requsições por serviço salvas em services/


### Tempo médio de request, proxy e kong por serviço

In [13]:
sqlEngine = create_engine(dbPath, pool_recycle=3600)
dbConnection = sqlEngine.connect()
querry='select \
            avg(sd.proxy) as "proxy", \
            avg(sd.kong) as "kong", \
            avg(sd.request) as "request", \
            sd.service_id as "service_id" \
        from \
            (select \
                r.index, \
                r.proxy, \
                r.kong, \
                r.request, \
                r.request_host, \
                s.service_id \
            from \
            test.requests as r \
            inner join test.services as s on r.service_id = s.index) as sd \
        group by sd.service_id'
pd.read_sql_query(querry, dbConnection).to_csv('services/medias_por_servico.csv')
dbConnection.close()
print('Tempo médio de request, proxy e kong por serviço salvo em services/')

Tempo médio de request, proxy e kong por serviço salvo em services/
